<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 

## 04 - Interoperability of the GPU PyData Ecosystem ##

**Table of Contents**
<br>
This notebook provides examples of how we can use cuDF and CuPy together to take advantage of CuPy array functionality (such as advanced linear algebra operations). This notebook covers the below sections: 
1. [NumPy, SciPy, and CuPy](#NumPy,-SciPy,-and-CuPy)
    * [cuDF vs. CuPy](#cuDF-vs.-CuPy)
2. [Working with CuPy](#Working-with-CuPy)
3. [Grid Converter](#Grid-Converter)
    * [Lat/Long to OSGB Grid Converter with NumPy](#Lat/Long-to-OSGB-Grid-Converter-with-NumPy)
    * [Lat/Long to OSGB Grid Converter with CuPy](#Lat/Long-to-OSGB-Grid-Converter-with-CuPy)
    * [Exercise #1 - Adding Grid Coordinate Columns to Dataframe](#Exercise-#1---Adding-Grid-Coordinate-Columns-to-DataFrame)
4. [Boolean Array Indexing](#Boolean-Array-Indexing)

## NumPy, SciPy, and CuPy ##
Per it's own user guide, [NumPy](https://numpy.org/doc/stable/user/whatisnumpy.html) is the fundamental package for scientific computing in Python. It is a Python library that provides a **multidimensional array object**, various derived objects (such as masked arrays and matrices), and an assortment of routines for fast operations on arrays. These operations include mathematical, logical, shape manipulation, sorting, selecting, I/O, discrete Fourier transforms, basic linear algebra, basic statistical operations, random simulation and much more. While NumPy focuses on arrays, mathematical operations, and basic linear algebra, [SciPy](https://docs.scipy.org/doc/scipy-1.8.1/tutorial/general.html) builds on this foundation to provide additional functionality, especially in the domain of scientific computing and optimization. 

On the other hands, [CuPy](https://cupy.dev/) is an open-source array library for GPU-accelerated computing with Python. CuPy can be seen as a GPU-accelerated counterpart to NumPy, offering similar functionality and API with the added benefit of GPU acceleration for compatible workloads. While NumPy operates on CPU memory, CuPy primarily works with GPU memory, leveraging CUDA-enabled GPUs for computation. CuPy's interface is highly compatible with NumPy and SciPy. In most cases it can be used as a drop-in replacement. All we need to do is just replace `numpy` and `scipy` with `cupy` and `cupyx.scipy` in the Python code. This makes it easier for users familiar with NumPy to transition to GPU-accelerated computing. 

CuPy is designed to work seamlessly with other GPU-accelerated libraries in the RAPIDS ecosystem, similar to how NumPy works with pandas and other CPU-based libraries. By keeping data on the GPU throughout the workflow, we are able to reduce data transfer overhead between CPU and GPU memory. 

### cuDF vs. CuPy ###
So far, the DataFrame we've worked with puts data in a structured, tabular format. This is useful when we need to perform DataFrame-like operations such as grouping, aggregating, filtering, and joining data. However, there might be use cases that requires working with multi-dimensional arrays or matrics, such as perfrming linear algebra operations or scientific computing tasks. For these instances, we would want to use libraries that are dedicated to performing these tasks, such as NumPy, SciPy, or CuPy. In other words, use cuDF when working with high-level (less abstract) data manipulation, and use CuPy when doing low-level numerical operations on multi-dimensional arrays. 

In practice, most data scientists work with both libraries, since most of their workflows involve DataFrame operations and array-based computations. For example, we might use cuDF for data loading and preprocessing, then convert to CuPy arrays for specific numerical computations, and convert back to cuDF for further analysis or output. cuDF and CuPy are designed to be interoperable, allowing us to easily convert between cuDF DataFrames/Series and CuPy arrays while keeping the data on the GPU. This enables us to create efficient workflows that take advntage of both libraries' strengths. 

## Working with CuPy ##
There are several ways to use CuPy. From cuDF, the `DataFrame.values` property will return the CuPy representation of the data frame. Alternatively, we can also convert via the CUDA array interface using `DataFrame.to_cupy()`. In addition to these, we can also pass the Series to the `cupy.asarray()` function since cuDF Series exposes the CUDA array interface as the fastest approach. 

Below we demonstrate a **row-wise sum** on the DataFrame. cuDF’s support for row-wise operations isn’t mature, so we’d need to either transpose the DataFrame or write a UDF and explicitly calculate the sum across each row. Transposing could lead to hundreds of thousands of columns (which cuDF wouldn’t perform well with) depending on our data’s shape, and writing a UDF can be time intensive. By leveraging the interoperability of the GPU PyData ecosystem, this operation becomes very easy. 

In [1]:
# DO NOT CHANGE THIS CELL
# import libraries
import cudf
import time

In [2]:
# DO NOT CHANGE THIS CELL
num_ele = 1000000

df = cudf.DataFrame(
    {
        "a": range(num_ele),
        "b": range(10, num_ele + 10),
        "c": range(100, num_ele + 100),
        "d": range(1000, num_ele + 1000)
    }
)

# preview
df.head()

,a,b,c,d
0,0,10,100,1000
1,1,11,101,1001
2,2,12,102,1002
3,3,13,103,1003
4,4,14,104,1004


In [3]:
# DO NOT CHANGE THIS CELL
start=time.time()
display(df.sum(axis=1))
time.time()-start

0            1110
1            1114
2            1118
3            1122
4            1126
           ...   
999995    4001090
999996    4001094
999997    4001098
999998    4001102
999999    4001106
Length: 1000000, dtype: int64

0.32047605514526367

The same operation runs faster with CuPy. 

In [4]:
# DO NOT CHANGE THIS CELL
arr=df.values

start=time.time()
# alternative approach
# arr=df.to_cupy()

display(arr.sum(axis=1))
time.time()-start

array([   1110,    1114,    1118, ..., 4001098, 4001102, 4001106])

0.21635150909423828

When using cuDF pandas, we can use the `.values` property as well as the `cupy.asarray()` function. 

**Note**: We can use the `.to_numpy()` method to convert cuDF DataFrames or Series to NumPy arrays. 

In [5]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# DO NOT CHANGE THIS CELL
%load_ext cudf.pandas
import pandas as pd

import numpy as np
import cupy as cp
import time

In [2]:
# DO NOT CHANGE THIS CELL
num_ele = 1000000

df = pd.DataFrame(
    {
        "a": range(num_ele),
        "b": range(10, num_ele + 10),
        "c": range(100, num_ele + 100),
        "d": range(1000, num_ele + 1000)
    }
)

# preview
df.head()

,a,b,c,d
0,0,10,100,1000
1,1,11,101,1001
2,2,12,102,1002
3,3,13,103,1003
4,4,14,104,1004


In [3]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
arr=df.values
# alternative approach
# arr=cp.asarray(df)

start=time.time()

display(arr.sum(axis=1))

time.time()-start

array([   1110,    1114,    1118, ..., 4001098, 4001102, 4001106])

                                                                       
                       Total time elapsed: 0.734 seconds               
                                                                       
                                     Stats                             
                                                                       
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                         ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 2        │     arr=df.values            │ 0.327483124 │             │
│          │                              │             │             │
│ 6        │     start=time.time()        │             │             │
│          │                              │             │             │
│ 8        │     display(arr.sum(axis=1)) │ 0.011984615 │             │
│          │                              │             │             │
│ 10       │     time.time()-start        │             │             │
│          │                              │             │             │
└──────────┴──────────────────────────────┴─────────────┴─────────────┘

Just like we can do with NumPy and pandas, we can weave cuDF and CuPy together in the same workflow while keeping the data entirely on the GPU. We’re able to seamlessly move between data structures in this ecosystem, giving us enormous flexibility without sacrificing speed. If we’re working with RAPIDS cuDF but need a more linear-algebra oriented function that exists in CuPy, we can leverage the interoperability of the GPU PyData ecosystem to use that function. 

To convert a CuPy array to a cuDF DataFrame or Series, we can use their respective constructors. 

In [4]:
# DO NOT CHANGE THIS CELL
df['sum']=arr.sum(axis=1)

df.head()

,a,b,c,d,sum
0,0,10,100,1000,1110
1,1,11,101,1001,1114
2,2,12,102,1002,1118
3,3,13,103,1003,1122
4,4,14,104,1004,1126


## Grid Converter ##
Much of our data is provided with latitude and longitude coordinates, but for some of our tasks involving distance - identifying geographically dense clusters of infected people, locating the nearest hospital or clinic from a given person - it is convenient to have Cartesian grid coordinates instead. By using a region-specific map projection - in this case, the [Ordnance Survey Great Britain 1936](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) - we can compute local distances efficiently and with good accuracy.

In [5]:
# DO NOT CHANGE THIS CELL
dtype_dict={
    'age': 'int8', 
    'sex': 'category', 
    'county': 'category', 
    'lat': 'float32', 
    'long': 'float32', 
    'name': 'category'
}
        
df=pd.read_csv('./data/uk_pop.csv', dtype=dtype_dict)
df.head()

,age,sex,county,lat,long,name
0,0,m,DARLINGTON,54.533638,-1.524400,FRANCIS
1,0,m,DARLINGTON,54.426254,-1.465314,EDWARD
2,0,m,DARLINGTON,54.555199,-1.496417,TEDDY
3,0,m,DARLINGTON,54.547909,-1.572342,ANGUS
4,0,m,DARLINGTON,54.477638,-1.605995,CHARLIE


### Lat/Long to OSGB Grid Converter with NumPy ###
To perform coordinate conversion, we will create a function `latlong2osgbgrid` which accepts latitude/longitude coordinates and converts them to [OSGB36 coordinates](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid): "northing" and "easting" values representing the point's Cartesian coordinate distances from the southwest corner of the grid.

Immediately below is `latlong2osgbgrid`, which relies heavily on NumPy:

In [6]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (north)
    long: longitude coordinate (east)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * np.pi/180
        long = long * np.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000                # northing of true origin
    E0 = 400000                 # easting of true origin
    F0 = .9996012717            # scale factor on central meridian
    phi0 = 49 * np.pi / 180     # latitude of true origin
    lambda0 = -2 * np.pi / 180  # longitude of true origin and central meridian
    
    sinlat = np.sin(lat)
    coslat = np.cos(lat)
    tanlat = np.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * np.sin(latdiff) * np.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * np.sin(2*(latdiff)) * np.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * np.sin(3*(latdiff)) * np.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - np.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

### Lat/Long to OSGB Grid Converter with CuPy ###
In the following `latlong2osgbgrid_cupy`, we simply swap `cp` in for `np`. While CuPy supports a wide variety of powerful GPU-accelerated tasks, this simple technique of being able to swap in CuPy calls for NumPy calls makes it an incredibly powerful tool to have at our disposal.

In [7]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (north)
    long: longitude coordinate (east)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000                 # northing of true origin
    E0 = 400000                  # easting of true origin
    F0 = .9996012717             # scale factor on central meridian
    phi0 = 49 * cp.pi / 180      # latitude of true origin
    lambda0 = -2 * cp.pi / 180   # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

Below we pass the latitude/longitude coordinates into the converter, which returns north and east values within the OSGB grid. 

In [8]:
%%time
# DO NOT CHANGE THIS CELL
numpy_lat = np.asarray(df['lat'])
numpy_long = np.asarray(df['long'])

CPU times: user 487 ms, sys: 560 ms, total: 1.05 s
Wall time: 1.05 s


In [10]:
%%time
# DO NOT CHANGE THIS CELL
n_numpy_array, e_numpy_array = latlong2osgbgrid(numpy_lat, numpy_long)

CPU times: user 12.9 s, sys: 3.3 s, total: 16.2 s
Wall time: 16.2 s


### Exercise #1 - Adding Grid Coordinate Columns to DataFrame ###
Now we will utilize `latlong2osgbgrid_cupy` to add `northing` and `easting` columns to `df`. We start by converting the two columns we need, `lat` and `long`, to CuPy arrays with the `cp.asarray()` function. Because cuDF and CuPy interface directly via the `__cuda_array_interface__`, the conversion can happen in nanoseconds. 
**Instructions**: <br>
* Execute the below cell to create CuPy arrays for the `lat` and `long` columns. 
* Modify the `<FIXME>` only and execute the cell below to use `latlong2osgbgrid_cupy` with `cupy_lat` and `cupy_long`, followed by add them as the `northing` and `easting` columns with the dtype `float32`. 

In [11]:
%%time
# DO NOT CHANGE THIS CELL
cupy_lat = cp.asarray(df['lat'])
cupy_long = cp.asarray(df['long'])

CPU times: user 929 μs, sys: 0 ns, total: 929 μs
Wall time: 942 μs


In [14]:
%%time
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
df['northing'] = pd.Series(n_cupy_array).astype('float32')
df['easting'] = e_cupy_array.astype('float32')
print(df.dtypes)
df.head()

age             int8
sex         category
county      category
lat          float32
long         float32
name        category
northing     float32
easting      float32
dtype: object
CPU times: user 3.54 s, sys: 49.1 ms, total: 3.59 s
Wall time: 3.57 s


,age,sex,county,lat,long,name,northing,easting
0,0,m,DARLINGTON,54.533638,-1.524400,FRANCIS,515491.90625,430772.15625
1,0,m,DARLINGTON,54.426254,-1.465314,EDWARD,503572.46875,434685.87500
2,0,m,DARLINGTON,54.555199,-1.496417,TEDDY,517903.65625,432565.53125
3,0,m,DARLINGTON,54.547909,-1.572342,ANGUS,517059.90625,427660.65625
4,0,m,DARLINGTON,54.477638,-1.605995,CHARLIE,509228.68750,425527.78125


Click ... for solution. 

## Boolean Array Indexing ##
Below we use `np.logical_and` for element-wise boolean selection.

In [13]:
# DO NOT CHANGE THIS CELL
start=time.time()
display(df.loc[np.logical_and(df['name'].str.startswith('E'), df['name'].str.endswith('D'))].head())
print(f'Duration: {round(time.time()-start, 2)} seconds')

,age,sex,county,lat,long,name
1,0,m,DARLINGTON,54.426254,-1.465314,EDWARD
49,0,m,DARLINGTON,54.575447,-1.600592,EDWARD
106,0,m,DARLINGTON,54.488045,-1.640927,EDWARD
145,0,m,DARLINGTON,54.492813,-1.509049,EDWARD
170,0,m,DARLINGTON,54.577919,-1.436109,EDWARD


Duration: 1.51 seconds


Below we use the CuPy for boolean selection. 

In [15]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
start=time.time()
display(df.loc[cp.logical_and(df['name'].str.startswith('E'), df['name'].str.endswith('D'))].head())
print(f'Duration: {round(time.time()-start, 2)} seconds')

,age,sex,county,lat,long,name,northing,easting
1,0,m,DARLINGTON,54.426254,-1.465314,EDWARD,503572.46875,434685.87500
49,0,m,DARLINGTON,54.575447,-1.600592,EDWARD,520113.18750,425816.00000
106,0,m,DARLINGTON,54.488045,-1.640927,EDWARD,510374.68750,423258.56250
145,0,m,DARLINGTON,54.492813,-1.509049,EDWARD,510957.09375,431797.09375
170,0,m,DARLINGTON,54.577919,-1.436109,EDWARD,520461.15625,436445.25000


Duration: 1.84 seconds


                                                                                                                   
                                             Total time elapsed: 1.934 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 2        │     start=time.time()                                                    │             │             │
│          │                                                                          │             │             │
│ 3        │     display(df.loc[cp.logical_and(df['name'].str.startswith('E'), df['n… │ 0.169462449 │ 1.218263904 │
│          │                                                                          │             │             │
│ 4        │     print(f'Duration: {round(time.time()-start, 2)} seconds')            │             │             │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

**Note**: String array is not yet implemented in CuPy. 

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** Let's move to the [next notebook](1-05_grouping.ipynb). 

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">